# InversePred (2023.04.12)

已建立好的模型進行反預測

  - y <- x +b
  - x <- rev_model(y)  
------------------------------

建立反預測方法，輸入目標值，基於模型可以提供對應的應變數
------------------------------
(2023.04.12)
> 先自行建好模型後，隨機生成數據，代入後得到數值，就可利用數值類似查表回推可能的變數組合。
> 建立模擬大表格，自變數(類似之前寫的基因突變與否)，個自變數生成範圍以真實數據做為參考，因此都要手動填每個參數想要的範圍
> 此模擬數據生成模組目前有三種是基於常態分佈抽樣而成，若為非連續值，則所有結果彼此機率相同。
> 
  - "g": 基於常態分佈生成，以平均值為分布中心(第一動差)，而標準差設定為1
  - "s": 基於樣本集所有的數值進行隨機抽樣，擴充數量。無法增加樣本集以外的數值，缺乏變異性。 
  - "int" or "seg": 等差級數，能自動偵測樣本集數值的小數點後位數，並以此為單位來創建等差級數。例如: 0.1, 0.2, 0.3。 


未來將新增
--------------------------------
  - 更多分布類型的偵測功能
  - 周邊的數值密讀，周邊的數值密度，以求更貼近目標值
  - 新增範例樣本、說明文件
  - 上傳到CRAN平台


Reference
------------------------------
- https://www.tandfonline.com/doi/abs/10.1080/03610918.2019.1601217




In [1]:

inverse_prediction <- function(model, target_value, simulated_data, return_number){
    predicted_values <- predict(model, simulated_data)
    simulated_data$pred_ <- predicted_values 
    ordered_<- simulated_data[order(abs(predicted_values-target_value)),]
    final_ <- ordered_[1:return_number,]
    close_true_value <- which.min(abs(predicted_values -target_value))
    return(list(final_))
 }



simulated_single_variable_value <- function(data, variables, size = 1000, mode="s", seg = 1){
    if(mode=="g"){
        #new_data <- sapply(seq_len(nrow(data)),function(i) sample(round(rnorm(1000, data[i,variables], 1), 2),size))
        new_data <- round(rnorm(size, mean(data[[variables]]), 1), 2)
    }else if(mode == "int"){
        new_data <- sample(seq(min(data[[variables]]), max(data[,variables]),seg),size,replace=TRUE)
    }else if(mode == "s"){
        new_data <- sample(data[[variables]], size, replace=TRUE)
    }
    new_data <- as.data.frame(new_data)
    colnames(new_data ) <- variables
    return(new_data)
}



sim_multi_vars_value <- function(data, variables, size, mode){
    seq_ <- lapply(c(variables), function(v) 1/10^max(sapply(data[[v]], decimalplaces)))
    new_data  <- lapply(1:length(variables), function(x) simulated_single_variable_value(data, variables[x], size , mode[x],seq_[[x]]))
    new_data <- as.data.frame(new_data)
    colnames(new_data ) <- variables
    return(new_data)
 }


decimalplaces <- function(x) {
    if ((x %% 1) != 0) {
        nchar(unlist(strsplit(sub('0+$', '', as.character(x)), ".", fixed=TRUE))[[2]])
    } else {
        return(0)
    }
}



# 測試區

In [5]:
library(chemCal)
library(inversePred)

m <- lm(y ~ x, data = massart97ex1)
result <- inverse.predict(m, 15)        #  6.1 +- 4.9
inverse_prediction(m, 15, simulated_data,1)
result$Prediction

$Prediction
[1] 6.09381

$`Standard Error`
[1] 1.767278

$Confidence
[1] 4.906751

$`Confidence Limits`
[1]  1.187059 11.000561

In [ ]:
simulated_data <- simulated_single_variable_value(training_dataset, "BDDE", 1000 ,"s", seg = 0.1)
simulated_data <- sim_multi_vars_value(training_dataset, c("BDDE","Size","Ratio"), 100000, c("s","int","s"))

result_ <- inverse_prediction(rf_model_1, 477, simulated_data, return_number = 5)


View(training_dataset)

